<a href="https://colab.research.google.com/github/agi2019/ppi-gci/blob/main/tutorials/02%20-%20model%20calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Model calibration</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>)

In this tutorial I will calibrate the free parameters of PPI's model. First, I will load all the data that I have prepared in the previous tutorials. Then, I extract the relevant information and put it in adequate data structures. Finally, I run the calibration function and save the results with the parameter values.

## Importing Python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np

## Importing PPI functions

In this tutorial, I will import the PPI source code directly from its repository. This means that I will place a request to GitHub, download the `policy_priority_inference.py` file, and copy it locally into the folder where these tutorials are saved. Then, I will import PPI. This approach is useful if you want to run this tutorial in a cloud computing service.

An alternative would be to manually copy the `policy_priority_inference.py` file into the folder where this tutorial is located.

In [2]:
import requests # the Python library that helps placing requests to websites
url = 'https://raw.githubusercontent.com/agi2019/ppi-gci/main/source_code/policy_priority_inference.py'
r = requests.get(url)
with open('policy_priority_inference.py', 'w') as f:
    f.write(r.text)
import policy_priority_inference as ppi

## Load data

### Indicators

In [3]:
df_indis = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_indicators.csv')

N = len(df_indis) # number of indicators
I0 = df_indis.I0.values # initial values
IF = df_indis.IF.values # final values
success_rates = df_indis.successRates.values # success rates
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix

### Interdependency network

In [4]:
df_net = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_network.csv')

A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

### Budget

In [5]:
df_exp = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_expenditure.csv')

Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

### Budget-indicator mapping

In [6]:
df_rela = pd.read_csv('https://raw.githubusercontent.com/agi2019/ppi-gci/main/tutorials/clean_data/data_relational_table.csv')

B_dict = {} # PPI needs the relational table in the form of a Python dictionary
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Calibrate

Now I run the calibration function to show that it works. Before that, let me explain a couple of new inputs that the user needs to provide:

* <strong>threshold</strong>: How well should the model be fit.
* <strong>parallel_processes</strong>: The number of processes (workers) to be ran in parallel.
* <strong>verbose</strong>: Whether to print or not the outputs as the calibration progresses.
* <strong>low_precision_counts</strong>: The number of iterations that use few Monte Carlo simulations.

The <strong>threshold</strong> parameter indicates the quality of the goodness of fit. More specifically, how good should the worst-fitted indicator be. The best possible fit is close to 1, but cannot be exactly 1 due to the stochasticity of the model. The higher the threshold, the mode Monte Carlo simulations are needed and, thus, the more time and computational resources are needed to complete the calibration.

Parameter <strong>parallel_processes</strong> is used to enhance efficiency. Since each Monte Carlo simulation is independent of each other, this workload can be distributed across multiple cores or processors. Today, most personal devices have the capability of handling this distributed load, so here I show how to calibrate the model using 4 parallel processes. It is recommended that you know how many cores or processors your equipment has, and that <strong>parallel_processes</strong> does not exceed that number. Otherwise, the performance of the calibration may be sub-optimal.

Finally, the <strong>low_precision_counts</strong> parameter helps accelerating the calibration. At the beginning of the calibration, the algorithm proposes a random set of parameters for the model. Because this proposal is unrelated to the true parameters, the errors tend to be large. In the presence of large errors, one can improve the goodness of fit without needing too much precision in each evaluation (i.e., without running too many Monte Carlo simulations). Hence, this parameter determines how many low-precision iterations of the algorithm should be run before proceeding to the high-precision ones. This accelerates the calibration procedure substantially.

In [7]:
T = Bs.shape[1]
parallel_processes = 4 # number of cores to use
threshold = 0.95 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 100 # number of low-quality iterations to accelerate the calibration

parameters = ppi.calibrate(I0, IF, success_rates, A=A, R=R, qm=qm, rl=rl, Bs=Bs, B_dict=B_dict,
              T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=True,
             low_precision_counts=low_precision_counts)

Iteration: 1 .    Worst goodness of fit: -148959952653.1602
Iteration: 2 .    Worst goodness of fit: -37005035938.16441
Iteration: 3 .    Worst goodness of fit: -7851271099.659472
Iteration: 4 .    Worst goodness of fit: -3141135374.5449605
Iteration: 5 .    Worst goodness of fit: -763752339.3344041
Iteration: 6 .    Worst goodness of fit: -228744351.47717816
Iteration: 7 .    Worst goodness of fit: -52139581.687649116
Iteration: 8 .    Worst goodness of fit: -9861377.90802407
Iteration: 9 .    Worst goodness of fit: -2664530.5682826987
Iteration: 10 .    Worst goodness of fit: -636477.33366552
Iteration: 11 .    Worst goodness of fit: -216034.1845025342
Iteration: 12 .    Worst goodness of fit: -31620.79177162709
Iteration: 13 .    Worst goodness of fit: -8883.351601329414
Iteration: 14 .    Worst goodness of fit: -3369.355387599554
Iteration: 15 .    Worst goodness of fit: -857.7803088085257
Iteration: 16 .    Worst goodness of fit: -359.0653612116966
Iteration: 17 .    Worst goodnes

## Calibration outputs

The output of the calibration function is a matrix with the following columns:

* <strong>alpha</strong>: the parameters related to structural constraints
* <strong>alpha_prime</strong>: the parameters related to structural costs
* <strong>beta</strong>: the parameters related to the probability of success
* <strong>T</strong>: the number of simulation periods
* <strong>error_alpha</strong>: the errors associated to the parameters $\alpha$ and $\alpha'$
* <strong>error_beta</strong>: the errors associated to the parameters $\beta$
* <strong>GoF_alpha</strong>: the goodness-of-fit associated to the parameters $\alpha$ and $\alpha'$
* <strong>GoF_beta</strong>: the goodness-of-fit associated to the parameters $\beta$

The top row of this matrix contains the column names, so I just need to transform these data into a DataFrame to export it.

In [8]:
df_params = pd.DataFrame(parameters[1::], columns=parameters[0])

In [9]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,0.0039793236390270265,8.570973949076493e-08,5.604023648021696e-09,60,-0.0027178547044797163,-0.0054362107077570165,0.9776270452083191,0.9942776729392031
1,0.008879507471659148,8.159646370723425e-08,1.0334448832220956e-08,nan,-0.0034999944657714277,-0.015753082287578102,0.9846776736692434,0.9834178081183388
2,0.010802090086519195,1.0500233841603905e-07,6.448096545898221e-09,nan,-0.0004239492352056118,0.010433700755290376,0.9986662478419349,0.9890171570996943
3,0.0028701619171869527,3.731628981325013e-07,5.239527050223303e-09,nan,0.002815439517975693,0.010679102767388482,0.9728115029412655,0.9887588391922226
4,2.786904289798832e-06,0.0034018398153559557,4.999931915116257e-10,nan,-0.0006099370807468585,-0.0004794746560643631,0.9967776323208078,0.9904105068787128
...,...,...,...,...,...,...,...,...
58,0.0012224813670157404,4.7534490422920535e-06,9.668522582007793e-09,nan,0.00024412286394137617,-0.003680906909231174,0.9931263235149068,0.9961253611481777
59,0.012186632616156998,0.001017650223950307,3.549106432803855,nan,-0.0003617879335650054,-0.00868993948061969,0.9993330284684061,0.9908526952835582
60,0.01030726513623305,2.001201819805822e-07,7.893419000581964e-09,nan,0.006421586148543046,0.02079394394755285,0.9837364701726816,0.9781116379499444
61,0.006559907562592159,7.411675873461397e-09,7.669436803248764e-09,nan,-0.00047340357630143437,-0.015688658492248098,0.998163756008071,0.9834856226397388


## Save parameters data

In [10]:
df_params.to_csv('parameters.csv', index=False)